In [25]:
#pip install geopy

In [26]:
#pip install folium

#### Step 1: Loading Libraries

In [27]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Step 2: Loading data from Wikipedia

In [28]:
request = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
Toronto  = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]
Toronto.rename(columns = {'Postcode':'PostalCode'},inplace=True)

In [29]:
Toronto.set_index('Borough').drop('Not assigned').reset_index(inplace=True)

#### Step 3: The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [30]:
Toronto.head(20) 

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


#### Step 4: More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [31]:
Toronto = Toronto.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

#### Step 5: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [32]:
Toronto.loc[Toronto['Neighbourhood']=='Not assigned','Neighbourhood'] = Toronto.loc[Toronto['Neighbourhood']=='Not assigned']['Borough']

##### Check the table to make sure Harbourfront and Regent Park are merged in M5A and Queen's Park is shown correctly at Neighbourhood.

In [33]:
## Table checking
Toronto.loc[Toronto['PostalCode']=='M5A']

,PostalCode,Borough,Neighbourhood
80,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [34]:
# Table checking
Toronto.loc[Toronto['Neighbourhood']=="Queen's Park"]

,PostalCode,Borough,Neighbourhood
120,M7A,Queen's Park,Queen's Park


##### Note: Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

### Step 6: In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [35]:
Toronto.shape

(180, 3)